In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#!pip install tensorflow==1.15rc2

In [ ]:
import requests 
from io import BytesIO
from PIL import Image

In [ ]:
style_img_path = "https://t1.daumcdn.net/cfile/blog/999EF93F5C8E10DE2E"

In [ ]:
content_img_path = "https://ifh.cc/g/Ospmt9.jpg" # 어떤 사진에다가 이미지를 입힐 것인가

In [ ]:
input_img = Image.open(BytesIO(requests.get(content_img_path).content))
input_img = input_img.resize((500,500))

In [ ]:
input_img

In [ ]:
style_img = Image.open(BytesIO(requests.get(style_img_path).content))
style_img = style_img.resize((500,500))

In [ ]:
style_img

In [ ]:
input_img_array = np.asarray(input_img,dtype='float32')


In [ ]:
input_img_array.shape

In [ ]:
input_img_array = np.expand_dims(input_img_array,axis=0) #axis=0 맨앞의 축을 늘려주겠다. (1,500,500,3)

In [ ]:
input_img_array.shape

In [ ]:
IMAGENET_MEAN_RGB_VALUES = [123.68, 116.779, 103.939] #BGR

In [ ]:
input_img_array[:,:,:,0] -= IMAGENET_MEAN_RGB_VALUES[2] #R채널
input_img_array[:,:,:,1] -= IMAGENET_MEAN_RGB_VALUES[1]
input_img_array[:,:,:,2] -= IMAGENET_MEAN_RGB_VALUES[0]

input_img_array = input_img_array[:,:,:,::-1]

In [ ]:
style_img_array = np.asarray(style_img,dtype='float32')
style_img_array = np.expand_dims(style_img_array,axis=0) #axis=0 맨앞의 축을 늘려주겠다. (1,500,500,3)
style_img_array[:,:,:,0] -= IMAGENET_MEAN_RGB_VALUES[2] #R채널
style_img_array[:,:,:,1] -= IMAGENET_MEAN_RGB_VALUES[1]
style_img_array[:,:,:,2] -= IMAGENET_MEAN_RGB_VALUES[0]

style_img_array = style_img_array[:,:,:,::-1]

In [ ]:
input_img_array

In [ ]:
from keras import backend

In [ ]:
input_img = backend.variable(input_img_array)

In [ ]:
input_img

In [ ]:
# np.min(input_img_array)
# #np.max(input_img_array)

In [ ]:
style_img = backend.variable(style_img_array)
#style_img = backend.variable(style_img)
style_img

In [ ]:
combination_img = backend.placeholder((1,500,500,3))

In [ ]:
input_tensor = backend.concatenate([input_img,style_img,combination_img],axis=0)

In [ ]:
input_tensor.shape

In [ ]:
from keras.applications.vgg16 import VGG16

In [ ]:
model = VGG16(input_tensor = input_tensor,include_top =False) #include_top:fcn층 안가져옴

mo

In [ ]:
model.summary()

In [ ]:
layers = dict([(x.name, x.output) for x in model.layers]) 

In [ ]:
layers

In [ ]:
#model.layers[0].output

In [ ]:
layer_features = layers['block2_conv2']

In [ ]:
layer_features

In [ ]:
#??
content_img_features = layer_features[0,:,:,:]
style_img_features = layer_features[1,:,:,:]
combination_img_features = layer_features[2,:,:,:]

In [ ]:
loss = backend.variable(0.)
loss

In [ ]:
def content_loss(content, combination):
    return backend.sum(backend.square(combination - content))

In [ ]:
def gram_matrix(x):
    # style을 어떻게 유지, 입힐 것인지. correlation계산? <구글 참고>
    features = backend.batch_flatten(backend.permute_dimensions(x, (2, 0, 1)))# flatten할 떄, permute 안 해주면 에러나서?   0=H. 1=W. 2=C
    gram = backend.dot(features, backend.transpose(features))
    return gram

def compute_style_loss(style, combination):
    # 두 개의 correlation을 같게 해주도록 하는 듯. style이니까, 이미지를 같게 해주는게 아닌, correlation정도를 같게 해주는 듯.
    style = gram_matrix(style)
    combination = gram_matrix(combination)
    size = 500 * 500
    return backend.sum(backend.square(style - combination)) / (4. * (3 ** 2) * (size ** 2))

In [ ]:
loss = loss+0.02*content_loss(content_img_features,combination_img_features )

In [ ]:
style_layers = ["block1_conv2", "block2_conv2", "block3_conv3", "block4_conv3", "block5_conv3"]
for layer_name in style_layers:
    layer_features = layers[layer_name]
    style_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    style_loss = compute_style_loss(style_features, combination_features)
    loss = loss + (4.5 / len(style_layers)) * style_loss

In [ ]:
ITERATIONS = 4
CHANNELS = 3
IMAGE_SIZE = 500
IMAGE_WIDTH = 500
IMAGE_HEIGHT = 500
IMAGENET_MEAN_RGB_VALUES = [123.68, 116.779, 103.939]
STYLE_WEIGHT = 4.5
TOTAL_VARIATION_WEIGHT = 0.995
TOTAL_VARIATION_LOSS_FACTOR = 1.25

def total_variation_loss(x): #규제 loss
    a = backend.square(x[:, :IMAGE_HEIGHT-1, :IMAGE_WIDTH-1, :] - x[:, 1:, :IMAGE_WIDTH-1, :])
    b = backend.square(x[:, :IMAGE_HEIGHT-1, :IMAGE_WIDTH-1, :] - x[:, :IMAGE_HEIGHT-1, 1:, :])
    return backend.sum(backend.pow(a + b, TOTAL_VARIATION_LOSS_FACTOR))

loss += TOTAL_VARIATION_WEIGHT * total_variation_loss(combination_img)

In [ ]:
outputs = [loss]
outputs += backend.gradients(loss, combination_img)

def evaluate_loss_and_gradients(x):
    x = x.reshape((1, IMAGE_HEIGHT, IMAGE_WIDTH, CHANNELS))
    outs = backend.function([combination_img], outputs)([x]) # loss, gradient 추출
    loss = outs[0]
    gradients = outs[1].flatten().astype("float64")
    return loss, gradients

class Evaluator:

    def loss(self, x):
        loss, gradients = evaluate_loss_and_gradients(x)
        self._gradients = gradients
        return loss

    def gradients(self, x):
        return self._gradients

evaluator = Evaluator()

In [ ]:
from scipy.optimize import fmin_l_bfgs_b

In [ ]:
%%time
x = np.random.uniform(0, 255, (1, IMAGE_HEIGHT, IMAGE_WIDTH, 3)) - 128.

for i in range(4):
    x, loss, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(), fprime=evaluator.gradients, maxfun=20)
    print("Iteration %d completed with loss %d" % (i, loss))
    
x = x.reshape((IMAGE_HEIGHT, IMAGE_WIDTH, CHANNELS))
x = x[:, :, ::-1]
x[:, :, 0] += IMAGENET_MEAN_RGB_VALUES[2]
x[:, :, 1] += IMAGENET_MEAN_RGB_VALUES[1]
x[:, :, 2] += IMAGENET_MEAN_RGB_VALUES[0]
x = np.clip(x, 0, 255).astype("uint8")
output_image = Image.fromarray(x)
output_image